# Ricerca: esercizi su espressioni regolari

Se non dobbiamo compiere operazioni *veramente* molto semplici con i metodi delle stringhe (`replace`, `search`, `index`, `upper`, `lower`, etc...) avrai bisogno di scrivere molto codice, lungo da leggere e da scrivere.

Se il tuo problema non è banale allora sicuramente avrai giovamento nell'utilizzare delle *espressioni regolari*.

## Prendiamoci i dati

Per partire da casi concreti, come già fatto in precedenza andiamo a cercarci dei dati dal catalogo opendata `dati.trentino.it`. In questo caso sceglieremo 


Il file [stops.txt](stops.txt) è un file CSV che contiene le informazioni riguardanti le stazioni degli autobus di Trento, ne mettiamo qui un estratto:

```
stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,wheelchair_boarding
1,28105z,Baselga Del Bondone,,46.078317,11.046924,10110,2
2,28105x,Baselga Del Bondone,,46.078581,11.047541,10110,2
3,27105c,Belvedere,,46.044406,11.105342,10110,2
4,22220z,Lamar Ponte Avisio,,46.134620,11.110914,10110,2
5,28060z,Sp 85 Bivio Sopramonte,,46.085226,11.069313,10110,2
7,24405z,Maso Bolleri,,46.102485,11.124174,10110,2
8,24405x,Maso Bolleri,,46.102234,11.123940,10110,2
9,25205x,Borino,,46.067367,11.165050,10110,2
```

Come ci aspettiamo da un buon file CSV, nella prima riga costituisce le intestazioni e vediamo che i campi sono separati da virgole.

## Verifichiamo che i dati siano corretti

Supponiamo che tu voglia fare avere a lista di tutte le fermate su *strade provinciali* (sigla *SP*): 

**DOMANDA**: Guarda bene dentro il [file completo](stops.txt): i dati sono sempre perfettamente regolari come ci piacerebbe? 

Una volta risposta alla domanda, pensiamo come fare questa operazione di ricerca di strade provinciali con Python, o almeno una prima approssimazione con questo codice: 

**DA FARE**: Con quello che sai [dalla lezione 2](../../data-formats-exercises.ipynb#File-CSV), prova ad aprire il CSV e stampare solo le linee che contengono qualcosa che assomiglia a strade provinciali. 

**NOTA**: non serve che filtri per bene tutte, fai solo qualche tentativo per i casi più ovvi, usando funzioni sulle stringhe che già conosci. Ai casi più diccifili ci penseremo in seguito con le regex! 

**SUGGERIMENTO 1**: usa il metodo `upper` delle stringhe:

In [3]:
"Ciao MONDO".upper()

'CIAO MONDO'

In [5]:
"SoftPython".upper()

'SOFTPYTHON'

**SUGGERIMENTO 2**: usa il metodo `find` delle stringhe (per maggiori info su come usarlo, usa `help(str.find)`)

In [19]:
"ab cde".find('cd')

3

In [20]:
"ab cde".find(' c')

2

In [22]:
"ab cde".find('a')

0

In [14]:
"ab cde".find('D')

-1

In [11]:
"ab cde".find('z')

-1


### Soluzione
1. Prima di tutto, bisogna aprire il file: dalla [lezione 2](../../data-formats.ipynb) sappiamo [come leggere](../../data-formats-exercises.ipynb#File-CSV) un file in formato CSV. Quindi apriamo, creiamo il lettore di CSV e lo utilizziamo per leggere tutte le righe nel file
2. per ogni riga possiamo normalizzare la terza colonna (ricorda che gli indici partono da zero) mettendola tutta maiuscola 
3. poi possiamo ricercare la posizione di `"SP"` all'interno del nome, per fare questo uso il metodo `str.find()` che ritorna la posizione della sottostringa, se questa non è presente ritorna `-1` 
4. infine stampare solo le righe che hanno passato il test.

In [16]:
import csv

with open('stops.txt', newline='') as f:
    reader = csv.reader(f, delimiter=',')    
    for row in reader:
        name = row[2].upper()
        if name.find("SP") >= 0:
            print(row)
        

['5', '28060z', 'Sp 85 Bivio Sopramonte', '', '46.085226', '11.069313', '10110', '2']
['46', '25305z', 'Gabbiolo "Campo Sportivo"', '', '46.055265', '11.148982', '10110', '1']
['47', '25305x', 'Gabbiolo "Campo Sportivo"', '', '46.055310', '11.148782', '10110', '1']
['48', '22080z', 'Sp 76 Carpenedi', '', '46.117195', '11.108678', '10110', '2']
['49', '22080x', 'Sp 76 Carpenedi', '', '46.117171', '11.108438', '10110', '2']
['104', '24040z', 'Sp 131 "Maso Pradiscola"', '', '46.083189', '11.135736', '10110', '2']
['105', '24040x', 'Sp 131 "Maso Pradiscola"', '', '46.083255', '11.135874', '10110', '1']
['109', '24350z', 'Sp 131 "Res. Silvana"', '', '46.097726', '11.126962', '10110', '2']
['110', '24350x', 'Sp 131 "Res. Silvana"', '', '46.097493', '11.126925', '10110', '2']
['115', '24355z', 'Sp 131 Al Maso Specchio', '', '46.104391', '11.123568', '10110', '2']
['116', '24355x', 'Sp 131 Al Maso Specchio', '', '46.104353', '11.123420', '10110', '2']
['131', '23040z', 'Sp 76 "Piac"', '', '46.

Il risultato non sembra quanto aspettato, `SP` si presenta in tante parole, e anche quando è riferito alle superstrade a volte è puntato e a volte no. Proviamo a vedere cosa sono queste **espressioni regolari**:

> **WIKIPEDIA**
>
> Una *espressione regolare* (in lingua inglese regular expression o, in forma abbreviata, regexp, regex o RE) è una sequenza di simboli (quindi una stringa) che identifica un insieme di stringhe: essa definisce una funzione che prende in ingresso una stringa, e restituisce in uscita un valore del tipo sì/no, a seconda che la stringa segua o meno un certo *pattern*.

L'utilizzo di una espressione regolare è sicuramente più veloce perché ci permette di cercare non solo una stringa bensì un intero insieme di stringe, detto appunto *pattern*. Una considerazione da fare è che nonostante i concetti
riguardo le regex sono universali, alcune implementazioni si differenziano nel comportamento in alcuni casi particolari oppure aggiungendo funzionalità *non standard*.


## Creiamoci una funzione di test

In ciò che segue, ci piacerebbe testare varie _regex_ su delle stringhe di prova. Dato che scrivere i comandi per stampare i risultati dei test può diventare ripetitivo e noiso, ci conviene creare una _funzione_  con dentro delle istruzioni di stampa da eseguire automaticamente (per più info sulle funzioni, vedi [capitolo 3 Pensare in Python ](https://davidleoni.github.io/ThinkPythonItalian/html/thinkpython2004.html)):

In [24]:
import re

def test_regex(pattern):
    names = ['Baselga Del Bondone', 
             'Sp.89 Maso Brentegam', 
             'sp 90 "Maso Prudenza"', 
             'Brancolino Sp.90']
    for name in names:
        print(re.search(pattern, name, re.I))
    
test_regex("SP")

<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='sp'>
<_sre.SRE_Match object; span=(11, 13), match='Sp'>



Ignoriamo per il momento `import re`, che semplicemente importa la libreria per le regex. 

In Python, le funzioni si dichiarano con la parola riservata `def` seguita dal nome della funzione che scegliamo arbitrariamente noi.  In questo caso il nome scelto è `test_regex`:

```python
def test_regex(pattern):
```
Poi, a questa funzione abbiamo deciso che bisognerà passare un parametro, che chiamiamo `pattern` (ma potremmo chiamarlo come ci pare, anche `pippo`).

**NOTA**: alla fine della prima riga, ci sono dei due punti **`:`** se dimentichi di metterli potresti trovarti con strani errori di syntassi ! 

La nostra funzione farà qualcosa con questa variabile che abbiamo chiamato `pattern`:

```python
def test_regex(pattern):
    names = ['Baselga Del Bondone',
             'Spini Bregenz',
             'Sp.89 Maso Brentegam', 
             'sp 90 "Maso Prudenza"', 
             'Brancolino Sp.90']
    for name in names:
        print(re.search(pattern, name, re.I))
```

In questo caso, per ciascun nome, eseguirà questa riga (vedremo in seguito il contenuto della `print`):

```python
print(re.search(pattern, name, re.I))
```

sfruttando la variabile `pattern` che passeremo al momento di chiamare la funzione: 

In [25]:
test_regex("SP")

<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='Sp'>
<_sre.SRE_Match object; span=(0, 2), match='sp'>
<_sre.SRE_Match object; span=(11, 13), match='Sp'>


Da questa chiamata, notiamo come un pattern si possa rappresentare con una semplice stringa Python, in questo caso `"SP"`

Andiamo nel dettaglio della funzione:

- creo una lista con le stringhe da testare: 

```python
    names = ['Baselga Del Bondone', 
             'Spini Bregenz',
             'Sp.89 Maso Brentegam', 
             'sp 90 "Maso Prudenza"', 
             'Brancolino Sp.90']
```

la prima `Baselga del Bondone` non è una strada provinciale, quindi ci aspettiamo che la ricerca di delle lettere 'sp' non abbia successo, questo è segnalato in Python dal ritornare  l'oggetto `None`. La successiva `Spini Bregenz` è più insidiosa, perchè contiene `Sp` pur non essendo una strada provinciale. Le altre invece vere strade provinciali con 'sp' nel nome e quindi dovrebbero ritornare un match.

- per ogni stringa nella lista stampo a schermo il risultato della chiamata al metodo `re.search()`:

```python
print(re.search(pattern, name, re.I))
```

Cosa prende `re.search` in ingresso?

    1. un `pattern` da cercare
    2. una `string` in cui cercarlo
    3. eventuali opzioni

Al momento dell'esecuzione:
    - se la ricerca ha successo: restiutisce un oggetto `SRE_Match` contenente le informazioni relative ala ricerca se la stringa soddisfa l'espressione regolare, 
    - se la ricerca fallisce: ritorna `None`
    
Come sicuramente avrai notato, usando il pattern `SP` il risultato è ancora sbagliato: infatti in questo caso è in tutto e per tutto equivalente a quanto fatto prima tramite il metodo `find()` sulle stringhe.


### DA FARE

Copia a mano la funzione di sopra qua sotto, ed eseguila con Control + Invio:

In [27]:
# scrivi qua

z

In [28]:
# scrivi qua la funzione senza `re.I`:


### La prima regex

È ora di usare la nostra prima regex, per farlo dobbiamo guardare le differenze tra le stringe corrette e quella errata: in questo caso sappiamo che ogni *strada provinciale* ha un numero. Proviamo con questa regex


In [18]:
test_regex(r"sp.\d\d")

None
<_sre.SRE_Match object; span=(0, 5), match='Sp.89'>
<_sre.SRE_Match object; span=(0, 5), match='sp 90'>
<_sre.SRE_Match object; span=(11, 16), match='Sp.90'>


Ora ti chiederai che cosa significa questa stringa, andiamo con ordine:
- La `r` prima dell'inizio della stringa serve ad indicare a python che la seguente è una * **r**aw string*, cioè una stringa un cui non deve espandere le *sequenze di escape* (cioè `\` seguito da altri caratteri al fine di generare caratteri non stampabili, per esempio `\n` è il carattere di new-line).
- Abbiamo visto che le lettere dell'alfabeto (e i numeri) hanno semplicemente il loro valore.
- Il carattere `.` in questo caso è un *metacarattere* che in questo caso si comporta come un "jolly" e può identificare qualsiasi carattere ad eccezione de carattere di fine riga (solitamente).
- `\d` sono due caratteri ma sono da considerarsi uno solo. Ogni volta che vediamo il carattere `\` è da considerarsi assieme al carattere successivo. Il significato in questo caso è una qualsiasi cifra tra `0` e `9`.

---

### DA FARE
Prova ad integrare questo esempio con quello precedente: copia il primo esempio qui sotto, aggiungi la libreria `re` e modifica *linea 7* perché l'*if* utilizzi la regular expression come abbiamo visto nell'esempio precedente.


In [19]:
with open('stops.txt', newline='') as f:
    reader = csv.reader(f, delimiter=',')    
    for row in reader:
        if re.search(r'sp.\d\d', row[2], re.I):
            print(row)

['5', '28060z', 'Sp 85 Bivio Sopramonte', '', '46.085226', '11.069313', '10110', '2']
['48', '22080z', 'Sp 76 Carpenedi', '', '46.117195', '11.108678', '10110', '2']
['49', '22080x', 'Sp 76 Carpenedi', '', '46.117171', '11.108438', '10110', '2']
['104', '24040z', 'Sp 131 "Maso Pradiscola"', '', '46.083189', '11.135736', '10110', '2']
['105', '24040x', 'Sp 131 "Maso Pradiscola"', '', '46.083255', '11.135874', '10110', '1']
['109', '24350z', 'Sp 131 "Res. Silvana"', '', '46.097726', '11.126962', '10110', '2']
['110', '24350x', 'Sp 131 "Res. Silvana"', '', '46.097493', '11.126925', '10110', '2']
['115', '24355z', 'Sp 131 Al Maso Specchio', '', '46.104391', '11.123568', '10110', '2']
['116', '24355x', 'Sp 131 Al Maso Specchio', '', '46.104353', '11.123420', '10110', '2']
['131', '23040z', 'Sp 76 "Piac"', '', '46.126690', '11.114532', '10110', '1']
['133', '23035z', 'Sp 76 "Maregioli"', '', '46.130755', '11.121750', '10110', '2']
['134', '23035x', 'Sp 76 "Maregioli"', '', '46.130836', '11.1

## Sintassi delle Python RegEx

Proviamo ora a guardare alcuni meta-caratteri importanti nelle regular expression in python (scusate per l'orribile tabella ma [il problema non è solo nostro](https://github.com/jupyter/notebook/issues/3024):

### Meta-caratteri

| Modificatore | Descrizione |
|:---:|:---|
| `\` | usato come *escape* (cioè segnalare che il carattere a seguire, nonostante sia un carattere speciale deve essere trattato come se non lo fosse) o per iniziare una *sequenza* (vedi sotto). |
| `.` | usato per rappresentare un qualsiasi carattere ad eccezione della nuova linea (con l'opzione `re.A` possiamo rimuovere anche questa eccezione). |
| `^` | usato per indicare l'inizio della riga |
| `$` | usato per indicare la fine della riga |
| `[...]` | usate per racchiudere l'insieme di caratteri che verificano questa espressione regolare |
| `[^...]` | usate per racchiudere l'insieme di caratteri che se presenti **NON** verificano questa espressione regolare |
| `A` &#124; `B` | usato per indicare una rappresentazione alternativa, è valida sia che appaia `A`, sia che appaia `B` |
| `()` | usate come in matematica per indicare la precedenza sulle operazioni |

Nel codice qui sotto ridefiniamo la funzione `test_regex()` usata in precedenza, utilizzando dei numeri di telefono al posto dei nomi delle fermate; per ogni numero controlliamo se l'espressione regolare viene soddisfatta e se lo è lo stampiamo ed in fine testiamo varie proprietà di questi numeri di telefono.

In [20]:
pattern = r''
def test_regex(pattern):
    numbers = ['3471234567', 
             '3303303367', 
             '3232123323', 
             '3383123222']
    for num in numbers:
        if re.search(pattern, num):
            print(num)
    print("-----")

print("Tutti i numeri che contengono 33")
test_regex("33")
print("Tutti i numeri che iniziano per 33")
test_regex("^33")
print("Tutti i numeri che hanno come penultima cifra 2")
test_regex("2.$")
print("Tutti i numeri che contengono 212 o 312")
test_regex("212|312")
# Oppure
test_regex("[23]12")
# Oppure
test_regex("(2|3)12")


Tutti i numeri che contengono 33
3303303367
3232123323
3383123222
-----
Tutti i numeri che iniziano per 33
3303303367
3383123222
-----
Tutti i numeri che hanno come penultima cifra 2
3232123323
3383123222
-----
Tutti i numeri che contengono 212 o 312
3232123323
3383123222
-----
3232123323
3383123222
-----
3232123323
3383123222
-----


### DA FARE
Prova a scrivere i pattern che soddisfano le proprietà richieste dalla stringa nella `print()` precedente. Cerca di *non guardare* all'esercizio precedente.

In [21]:
print("Tutti i numeri che contengono 32")
test_regex("32")

print("Tutti i numeri che finiscono per 67")
test_regex("67$")

print("Tutti i numeri che hanno come quarta cifra 3")
test_regex("^...3")

print("Tutti i numeri che contengono 232 o 233 o 234")
test_regex("232|233|234")
# Oppure
test_regex("23[234]")
# Oppure
test_regex("23(2|3|4)")
# Oppure
#test_regex("23[2-4]")

Tutti i numeri che contengono 32
3232123323
3383123222
-----
Tutti i numeri che finiscono per 67
3471234567
3303303367
-----
Tutti i numeri che hanno come quarta cifra 3
3303303367
3383123222
-----
Tutti i numeri che contengono 232 o 233 o 234
3471234567
3232123323
3383123222
-----
3471234567
3232123323
3383123222
-----
3471234567
3232123323
3383123222
-----


### Ripetizioni

Le espressioni regolari possono anche gestire delle ripetizioni di particolari pattern utilizzando altri caratteri speciali. 

| Modificatori | Descrizione |
| :---: | :--- |
| `{m, n}` | il carattere o gruppo a cui è riferito viene ripetuto almeno `m` volte fino ad un massimo di `n` volte. |
| `{m}` | il carrattere o il gruppo a cui è riferito viene ripetuto esattamente `m` volte |
| `?` | il carattere o il gruppo a cui è riferito viene ripetuto `0` o `1` volta. Equivale a `{,1}`. |
| `*` | il carattere o il gruppo a cui è riferito viene ripetuto `0` o più volte . Equivale a `{,}`. |
| `+` | il carattere o il gruppo a cui è riferito viene ripetuto `1` o più volte. Equivale a `{1,}`. |

I caratteri o gruppi a cui si riferiscono i `modificatori delle ripetizioni` appena precedenti ad essi, vediamo un esempio

In [22]:
print(re.search("a+b", "aaaabbb"))
print(re.search("a+b", "bab"))
print(re.search("a+b", "bbb"))
print(re.search("a+b", "aaaa"))

<_sre.SRE_Match object; span=(0, 5), match='aaaab'>
<_sre.SRE_Match object; span=(1, 3), match='ab'>
None
None


Come pui vedere nell'esempio qui sopra il pattern `a+b` indica `a` una o più volte, seguito da una `b`. Quano si ha un match puoi vedere nell'oggetto ritornato dal metodo `re.search()` la sottostringa che ha verificato la regex (usando il metodo `group()`) e gli indici della posizione di essa all'interno della stringa (chiamando il metodo `.span()`).

### DA FARE

Qui sotto la funzione `pass_n_fail()` prende come parametri un `pattern` e due liste di stringhe: `pass_list` e `fail_list`. La funzione controlla se quelle che appartengono alla prima lista verifichino l'espressione regolare mentre quelle nella seconda non lo facciano. 

Prova a completare il codice aggiungendo delle stringhe (almeno 2) che soddisfano i pattern assegnati:

In [23]:
def pass_n_fail(pattern, pass_list, fail_list):
    for p in pass_list:
        if not re.search(pattern, p):
            print("ERRORE: '{}' non verifica {} e dovrebbe farlo".format(p, pattern))
    for f in fail_list:
        if re.search(pattern, f):
            print("ERRORE: '{}' verifica {} e non dovrebbe farlo".format(f, pattern))

# ESEMPIO
pass_n_fail("a",
            ["aa","a"],
            ["b","c"]
           )
# Prova tu:
pass_n_fail("a",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non verifica a e dovrebbe farlo
ERRORE: '' non verifica a e dovrebbe farlo


In [24]:
# Adesso un po più difficile
pass_n_fail("ab+",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non verifica ab+ e dovrebbe farlo
ERRORE: '' non verifica ab+ e dovrebbe farlo


In [25]:
# Adesso un po più difficile ancora
pass_n_fail("ab*",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non verifica ab* e dovrebbe farlo
ERRORE: '' non verifica ab* e dovrebbe farlo


In [26]:
# OK! Ancora un paio
pass_n_fail("^[ab]+[^ab]$",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non verifica ^[ab]+[^ab]$ e dovrebbe farlo
ERRORE: '' non verifica ^[ab]+[^ab]$ e dovrebbe farlo


In [27]:
# L'ultima
pass_n_fail(".?\.{3}$",
            ["",""], #TODO
            ["",""]
           )

ERRORE: '' non verifica .?\.{3}$ e dovrebbe farlo
ERRORE: '' non verifica .?\.{3}$ e dovrebbe farlo


### Sequenze

A volte vogliamo considerare insiemi molto grandi di possibili simboli in una espressione regolare: per esempio se vogliamo validare la struttura di un indirizzo email vogliamo controllare che contenga almeno 3 caratteri, una `@` altri 3 caratteri un punto e almeno altri 2 caratteri. Il problema è che alcuni caratteri non possono essere presenti nelle email (come ad esempio `\|{}()[]` etc...), se dovessimo scrivere un set di caratteri da escludere usando l'espressione `[^...]` ci costerebbe molto tempo e spazio, inoltre sarebbe facile dimenticarsi qualche simbolo e quasi impossibile da leggere. Per ovviare a questo problema sono stati introdotte delle scorciatoie: delle *sequenze* di simboli che vanno a sostituire lunghi set di caratteri di comune utilizzo, eccone alcuni:

| Sequenza | Descrizione |
| :---: | :--- |
| \A | Inizio della stringa (simile a `^`) |
| \d | Cifre da 0 a 9 |
| \D | Tutto eccetto le cifre |
| \s | Spaziature |
| \S | Tutto eccetto le spaziature |
| \w | Tutti i caratteri alfanumerici e `_` |
| \W | Tutto eccetto i caratteri alfanumerici e l' `_` |
| \Z | Fine della stringa (simile a `$`) |

### DA FARE

Proviamo a variare l'esercizio precedente adesso quando macherà sarai tu a *scrivere dei pattern* che verifichi le stringhe nella `pass_list` ed escluda quelle nella `fail_list`, dove troverai il pattern sarà come l'esercizio precedente.


In [32]:
#Scegli il PATTERN
pass_n_fail(r"^\d\s\w+",
            ["3 ramarri", "2 carri"],
            ["tre ramarri", "due carri", "3 ", "2 "]
           )

In [33]:
#Scegli le STRINGHE
pass_n_fail(r"^(w{3}\.)?\w\w+\.\w\w+",
            ["www.google.it","youtube.com"],
            ["ciaociao"," www.google.com"]
           )

In [35]:
#Scegli il PATTERN
pass_n_fail(r"\d{2}[-/.][01]\d[-/.]\d\d(\d\d)?$",
            ["21.12.2017", "11/01/2018", "16-12-89"],
            ["1621211003", "11/20/2010", "12-12-123"]
           )

## Le funzioni della libreria `re`

Fino ad ora abbiamo usato una sola funzione della libreria `re` di Python, ossia `re.search()` ma sono presenti anche altre funzionalità, la più simile è il metodo `re.match()`:


In [30]:
print(re.search('c', 'abcde'))
print(re.match('c', 'abcde'))
print(re.match('a', 'abcde'))

<_sre.SRE_Match object; span=(2, 3), match='c'>
None
<_sre.SRE_Match object; span=(0, 1), match='a'>


Riesci a capire la differenza? `help(re.match)` e `help(re.search)` possono tornarti utili. 